In [1]:
import pandas as pd

In [2]:
file_path = "C:/Users/laeti/Documents/laetitia/EPFL/Master/CoursMA2/Data visualization/dataLaeti/dataMedails.xlsx"
# Read data from Excel sheets
df_men = pd.read_excel(file_path, sheet_name="MenTopCountry", header=0)
df_women = pd.read_excel(file_path, sheet_name="WomanTopCountry", header=0)

In [9]:
# Combine the dataframes
df_combined = pd.concat([df_men, df_women])

# Group by Nation and sum the medals
df_total_medals = df_combined.groupby('Nation', as_index=False).sum()

# Calculate the total medals
df_total_medals['Total'] = df_total_medals['Gold'] + df_total_medals['Silver'] + df_total_medals['Bronze']

# Sort by total medals (optional)
df_total_medals = df_total_medals.sort_values(by='Total', ascending=False).reset_index(drop=True)

# Function to remove abbreviation
def remove_abbreviation(country_name):
    return re.sub(r'\s*\(.*?\)\s*', '', country_name)

file_path_bis = "C:/Users/laeti/Documents/laetitia/EPFL/Master/CoursMA2/Data visualization/dataLaeti/medals_per_contry_total.xlsx"
df_total_medals.to_csv(file_path_bis, index=False)
# Display the resulting dataframe
print(df_total_medals)

                                Nation  Gold  Silver  Bronze  Total
0                                  USA     3       7       4     14
1                       United Kingdom     5       5       4     14
2                               Kenya      7       3       2     12
3                             Germany      0       5       5     10
4                              Russia      4       2       2      8
..                                 ...   ...     ...     ...    ...
263                             Guyana     0       0       0      0
264                              Haiti     0       0       0      0
265  Heard Island and McDonald Islands     0       0       0      0
266                           Honduras     0       0       0      0
267                           Zimbabwe     0       0       0      0

[268 rows x 5 columns]


In [4]:
df_women_full = pd.read_excel(file_path, sheet_name="WomanMedalists Full", header=0)
df_men_full = pd.read_excel(file_path, sheet_name="MenMedalistsFull", header=0)

In [5]:
import pandas as pd

def add_winner_names(df_medals, df_winners):
    # Initialize empty lists to store the winner names
    gold_winners = []
    silver_winners = []
    bronze_winners = []
    
    # Iterate over each row in the medals DataFrame
    for index, row in df_medals.iterrows():
        year = row['Year']
        
        # Find the corresponding row in the winners DataFrame based on the year
        winner_row = df_winners[df_winners['Games'] == year].iloc[0]
        
        # Extract winner names for each medal type only if the count is non-zero
        gold_winner = winner_row['GoldWinner'] if row['Gold count'] > 0 else None
        silver_winner = winner_row['SilverWinner'] if row['Silver count'] > 0 else None
        bronze_winner = winner_row['BronzeWinner'] if row['Bronze count'] > 0 else None
        
        # Append winner names to the lists
        gold_winners.append(gold_winner)
        silver_winners.append(silver_winner)
        bronze_winners.append(bronze_winner)
    
    # Add winner names to the original DataFrame
    df_medals['Gold Winner'] = gold_winners
    df_medals['Silver Winner'] = silver_winners
    df_medals['Bronze Winner'] = bronze_winners
    
    return df_medals


In [6]:
def medail_per_year(df_gender):

    # Melt the dataframe to long format
    df_melted = pd.melt(df_gender, id_vars=['Games', 'Location'], value_vars=['Gold', 'Silver', 'Bronze'], var_name='Medal', value_name='Country')
    # Sort by year and country
    df_melted = df_melted.sort_values(by=['Country', 'Games'])

    # Pivot the melted dataframe
    df_pivoted = df_melted.pivot_table(index=['Country', 'Games'], columns='Medal', aggfunc='size', fill_value=0).reset_index()

    # Rename columns for clarity
    df_pivoted.columns = ['Country', 'Year','Gold count', 'Silver count', 'Bronze count' ]    
    df_pivoted = df_pivoted[['Country', 'Year','Silver count', 'Bronze count', 'Gold count' ]]  # Swapping columns because shady classification
    df_pivoted.columns = ['Country', 'Year','Gold count', 'Silver count', 'Bronze count' ]
    
    return add_winner_names(df_pivoted,df_gender )

In [7]:
medail_year_men = medail_per_year(df_men_full)
medail_year_women=medail_per_year(df_women_full)